# Preprocessing and Cleaning

In [0]:
#Necessary imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [0]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [0]:
housing = pd.read_csv('train.csv')

In [0]:
housing_predict = pd.read_csv('test.csv')

In [0]:
housing.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Drops the union of the columns that have null values in both the test and training data sets

In [0]:
housing_null_columns = [i for i in housing.columns if housing[i].isnull().any()]
housing.dropna(axis=1, inplace=True)

In [0]:
housing_predict_null_columns = [i for i in housing_predict.columns if housing_predict[i].isnull().any()]

In [0]:
housing = housing.drop([x for x in housing_predict_null_columns if x not in housing_null_columns], axis=1)

In [0]:
 #Seperate quantitative and categorical features in training
 housing_quantitative = housing[housing.select_dtypes(include='int64').columns]
 housing_categorical = housing[housing.select_dtypes(include='object').columns]

In [0]:
 #Seperate quantitative and categorical features in testing
housing_predict.dropna(axis=1, inplace=True)
housing_predict = housing_predict.drop([x for x in housing_null_columns if x not in housing_predict_null_columns], axis=1)
housing_predict_quantitative = housing_predict[housing_predict.select_dtypes(include='int64').columns]
housing_predict_categorical = housing_predict[housing_predict.select_dtypes(include='object').columns]

In [0]:
housing_predict.isna().sum()

Id               0
MSSubClass       0
LotArea          0
Street           0
LotShape         0
LandContour      0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
RoofMatl         0
ExterQual        0
ExterCond        0
Foundation       0
Heating          0
HeatingQC        0
CentralAir       0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
TotRmsAbvGrd     0
Fireplaces       0
PavedDrive       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
MiscVal          0
MoSold           0
YrSold           0
SaleCondition    0
dtype: int64

In [0]:
#Assigns dummy variables for categorical features
housing_categorical_dummies = pd.get_dummies(housing_categorical,drop_first=True)
housing_predict_categorical_dummies = pd.get_dummies(housing_categorical,drop_first=True)

In [0]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(housing_quantitative)  
housing_quantitative_scaled = pd.DataFrame(scaler.transform(housing_quantitative), columns = housing_quantitative.columns)
#housing_predict_quantitative_scaled = pd.DataFrame(scaler.transform(housing_predict_quantitative), columns = housing_predict_quantitative.columns)

In [0]:
#Scales quantitative data
scaler_predict = StandardScaler()  
scaler_predict.fit(housing_predict_quantitative)  
housing_predict_quantitative_scaled = pd.DataFrame(scaler_predict.transform(housing_predict_quantitative), columns = housing_predict_quantitative.columns)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [0]:
from sklearn import metrics
from sklearn.metrics import r2_score

In [0]:
#Unscaled X data inputs for linear regression
X_train = pd.concat([housing_quantitative.drop(['SalePrice', 'Id'], axis=1), housing_categorical_dummies], axis=1)
X_test = pd.concat([housing_predict_quantitative.drop(['Id'], axis=1), housing_predict_categorical_dummies], axis=1)
X_test.dropna(axis=0, inplace=True)

In [0]:
X_train.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,Street_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,...,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,PavedDrive_P,PavedDrive_Y,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,8450,7,5,2003,2003,856,854,0,1710,2,1,3,1,8,0,0,61,0,0,0,0,0,2,2008,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0
1,20,9600,6,8,1976,1976,1262,0,0,1262,2,0,3,1,6,1,298,0,0,0,0,0,0,5,2007,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0
2,60,11250,7,5,2001,2002,920,866,0,1786,2,1,3,1,6,1,0,42,0,0,0,0,0,9,2008,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0
3,70,9550,7,5,1915,1970,961,756,0,1717,1,0,3,1,7,1,0,35,272,0,0,0,0,2,2006,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0
4,60,14260,8,5,2000,2000,1145,1053,0,2198,2,1,4,1,9,1,192,84,0,0,0,0,0,12,2008,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0


In [0]:
X_test.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,Street_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,...,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,PavedDrive_P,PavedDrive_Y,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,20.0,11622.0,5.0,6.0,1961.0,1961.0,896.0,0.0,0.0,896.0,1.0,0.0,2.0,1.0,5.0,0.0,140.0,0.0,0.0,0.0,120.0,0.0,0.0,6.0,2010.0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0
1,20.0,14267.0,6.0,6.0,1958.0,1958.0,1329.0,0.0,0.0,1329.0,1.0,1.0,3.0,1.0,6.0,0.0,393.0,36.0,0.0,0.0,0.0,0.0,12500.0,6.0,2010.0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0
2,60.0,13830.0,5.0,5.0,1997.0,1998.0,928.0,701.0,0.0,1629.0,2.0,1.0,3.0,1.0,6.0,1.0,212.0,34.0,0.0,0.0,0.0,0.0,0.0,3.0,2010.0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0
3,60.0,9978.0,6.0,6.0,1998.0,1998.0,926.0,678.0,0.0,1604.0,2.0,1.0,3.0,1.0,7.0,1.0,360.0,36.0,0.0,0.0,0.0,0.0,0.0,6.0,2010.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0
4,120.0,5005.0,8.0,5.0,1992.0,1992.0,1280.0,0.0,0.0,1280.0,2.0,0.0,2.0,1.0,5.0,0.0,0.0,82.0,0.0,0.0,144.0,0.0,0.0,1.0,2010.0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0


In [0]:
#y data input
y_train = housing['SalePrice']

# Linear Regression

**RMSLE = 0.50824**

In [0]:
LinRegression = LinearRegression()
LinRegression.fit(X_train, y_train)

predictions = LinRegression.predict(X_test)

In [0]:
predictionsdf = pd.DataFrame(predictions)
predictionsdf.head()

,0
0,124178.312719
1,169838.792226
2,184213.105345
3,198138.092639
4,227230.066150


In [0]:
predictionsdf.to_csv('Predictions.csv')

# MLP Regressor

**RMSLE = 0.24452**

In [0]:
from sklearn.neural_network import MLPRegressor

X_train = pd.concat([housing_quantitative_scaled.drop(['SalePrice', 'Id'], axis=1), housing_categorical_dummies], axis=1)
X_test = pd.concat([housing_predict_quantitative_scaled.drop(['Id'], axis=1), housing_predict_categorical_dummies], axis=1)
X_test.dropna(axis=0, inplace=True)

MLP = MLPRegressor(alpha=0.00005, hidden_layer_sizes=200, max_iter=5000)
MLP.fit(X_train, y_train)

predictions_MLP = pd.DataFrame(MLP.predict(X_test))
predictions_MLP.head()

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


,0
0,129406.413518
1,151019.361415
2,179959.398421
3,199525.279387
4,208395.747902


In [0]:
predictions_MLP.to_csv('PredictionsMLPScaled5000.csv')

# RidgeCV

**RMSLE = 0.25711**

In [0]:
from sklearn.linear_model import RidgeCV

rcv = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1])
rcv.fit(X_train, y_train)

predictions_rcv = pd.DataFrame(rcv.predict(X_test))
predictions_rcv.head()

,0
0,123620.531296
1,164458.097318
2,186309.110748
3,199008.975960
4,221806.265482


In [0]:
predictions_rcv['Id'] = predictions_rcv.index
predictions_rcv = predictions_rcv[['Id', 0]]
predictions_rcv['Id'] = predictions_rcv['Id'].apply(lambda x: x + 1461)
predictions_rcv.rename(columns={0:'SalePrice'}, inplace=True)

In [0]:
predictions_rcv.head()

,Id,SalePrice
0,1461,123620.531296
1,1462,164458.097318
2,1463,186309.110748
3,1464,199008.975960
4,1465,221806.265482


In [0]:
predictions_rcv.tail()

,Id,SalePrice
1454,2915,141390.454663
1455,2916,84951.759268
1456,2917,156348.896985
1457,2918,157359.948539
1458,2919,245393.328536


In [0]:
predictions_rcv.to_csv('PredictionsRCV.csv')

# KERAS Sequential

**RMSLE = 0.21497**

In [0]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [0]:
X_train.keys()

Index(['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       ...
       'HeatingQC_Po', 'HeatingQC_TA', 'CentralAir_Y', 'PavedDrive_P',
       'PavedDrive_Y', 'SaleCondition_AdjLand', 'SaleCondition_Alloca',
       'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=129)

In [0]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(X_train.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [0]:
model = build_model()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 1000

history = model.fit(
  X_train, y_train,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])


....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
..........................................................................................

In [0]:
predictions_keras = pd.DataFrame(model.predict(X_test).flatten())
predictions_keras['Id'] = predictions_keras.index
predictions_keras = predictions_keras[['Id', 0]]
predictions_keras['Id'] = predictions_keras['Id'].apply(lambda x: x + 1461)
predictions_keras.rename(columns={0:'SalePrice'}, inplace=True)

In [0]:
predictions_keras.head()

,Id,SalePrice
0,1461,135549.781250
1,1462,162388.734375
2,1463,185841.640625
3,1464,189639.796875
4,1465,197631.703125


In [0]:
predictions_keras.to_csv('PredictionsKeras.csv')